pipeline：
1. 指定一个notion page（ubuntu的操作命令、conda pip的命令等），通过自定义的Loader加载；
2. 分割成13个片段，每个大小为1000个token，两两之间重叠部分为200个token；
3. 通过开源的BAAI/bge-m3或bge-large-zh-v1.5，对片段进行向量化，并存储在FAISS（向量相似性搜索库）中；
4. 设计RAG prompt和RAG chain；
5. 测试GPT在有无RAG情况下的表现。

In [1]:
from notion_client import Client
import os
from dotenv import load_dotenv
load_dotenv() # 在当前路径下新建一个.env文档，其中存储了代理信息

NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_PAGE_ID = os.getenv("NOTION_PAGE_ID")

notion = Client(auth=NOTION_API_KEY)
page_id = NOTION_PAGE_ID
page_content = notion.blocks.children.list(block_id=page_id)

In [8]:
print(page_content.get("results")[0].get('parent').get('page_id'))

ee04ec75-c67b-460b-9aa0-b2143cead3a2


以一个ubuntu 20.04 和 22.04 命令、操作汇总的notion笔记为例。  
notion的page，是由各种block组成：
1. 多种 block 类型处理：不同的 block 类型（如列表、段落、折叠块等）需要不同的方式提取文本。
2. 嵌套结构：子页面和同步块可能包含更深层次的内容，需要递归获取。

例子：
1. 第一个block是bulleted_list_item，在notion中是前面带·
2. 第二个是paragraph，其rich_text是空，再notion中是空白行
3. 第三个是synced_block，是同步块，提供了被同步的块id，需要迭代
4. 第四个是toggle，是个折叠块，折叠的内容得通过block id请求notion API的方式获取
5. 最后如果页面太大，会分页：has_more、next_cursor

```json
{'object': 'list',
 'results': [{'object': 'block',
   'id': 'xxx',
   'parent': {'type': 'page_id',
    'page_id': 'xxx'},
   'created_time': 'xxx',
   'last_edited_time': '2025-03-06T13:49:00.000Z',
   'created_by': {'object': 'user',
    'id': 'xxx'},
   'last_edited_by': {'object': 'user',
    'id': 'xxx'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'bulleted_list_item',
   'bulleted_list_item': {'rich_text': [{'type': 'text',
      'text': {'content': 'Ubuntu Oracular 24.10', 'link': None},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'default'},
      'plain_text': 'Ubuntu Oracular 24.10',
      'href': None}],
    'color': 'default'}},
  {'object': 'block',
   'id': 'xxx',
   'parent': {'type': 'page_id',
    'page_id': 'xxx'},
   'created_time': 'xxx',
   'last_edited_time': 'xxx',
   'created_by': {'object': 'user',
    'id': 'xxx'},
   'last_edited_by': {'object': 'user',
    'id': 'xxx'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [], 'color': 'default'}},
  {'object': 'block',
   'id': 'xxx',
   'parent': {'type': 'page_id',
    'page_id': 'xxx'},
   'created_time': 'xxx',
   'last_edited_time': 'xxx',
   'created_by': {'object': 'user',
    'id': 'xxx'},
   'last_edited_by': {'object': 'user',
    'id': 'xxx'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'synced_block',
   'synced_block': {'synced_from': {'type': 'block_id',
     'block_id': 'xxx'}}},
  {'object': 'block',
   'id': 'xxx',
   'parent': {'type': 'page_id',
    'page_id': 'xxx'},
   'created_time': 'xxx',
   'last_edited_time': 'xxx',
   'created_by': {'object': 'user',
    'id': 'xxx'},
   'last_edited_by': {'object': 'user',
    'id': 'xxx'},
   'has_children': True,
   'archived': False,
   'in_trash': False,
   'type': 'toggle',
   'toggle': {'rich_text': [{'type': 'text',
      'text': {'content': '笔记本盒盖时行为设置', 'link': None},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'default'},
      'plain_text': '笔记本盒盖时行为设置',
      'href': None}],
    'color': 'default'}}],
 'next_cursor': 'xxx',
 'has_more': True,
 'type': 'block',
 'block': {},
 'request_id': 'xxx'}
 ```

In [9]:
def get_all_text(page_id):
    text = ""
    # 根据page_id，从notion api中获取页面
    response = notion.blocks.children.list(block_id=page_id)

    while True:
        for block in response.get("results", []):
            # 提取当前 Block 文本
            block_type = block["type"]
            content = block[block_type]
            if "rich_text" in content:
                for rt in content["rich_text"]:
                    text += rt.get("text", {}).get("content", "") + "\n"
            # 递归处理子 Blocks
            if block.get("has_children"):
                text += get_all_text(block["id"])
        # 处理分页
        if not response.get("has_more"):
            break
        response = notion.blocks.children.list(
            block_id=page_id,
            start_cursor=response["next_cursor"]
        )
    return text

page_text = get_all_text(page_id)
# 2min

In [10]:
from langchain.docstore.document import Document

documents = []
documents.append(Document(page_content=page_text, metadata={"source": page_id}))

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 块大小
    chunk_overlap=200  # 块间重叠
)
splits = text_splitter.split_documents(documents)

In [12]:
print(len(splits))

13


In [13]:
# 下载 Embedding 模型，从 HuggingFace 拉取模型至本地

# ! cd models/sentence_embedding
# ! git clone https://huggingface.co/BAAI/bge-m3
# ! git clone https://huggingface.co/BAAI/bge-large-zh
# ! git clone https://huggingface.co/BAAI/bge-large-zh-v1.5

In [14]:
# https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html

# from langchain.embeddings import HuggingFaceEmbeddings # 已淘汰
from langchain_huggingface import HuggingFaceEmbeddings

# 本地：models/sentence_embedding/bge-large-zh、bge-large-zh-v1.5、bge-m3
# 从hf获取：BAAI/bge-large-zh、BAAI/bge-large-zh-v1.5、BAAI/bge-m3
model_name = "../models/sentence_embedding/bge-large-zh-v1.5"
model_kwargs = {'device': 'cuda'}
# normalize_embeddings （ bool ，可选）– 是否将返回的向量标准化为长度 1。在这种情况下，可以使用更快的点积（util.dot_score）而不是余弦相似度。默认为 False。
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/hey/anaconda3/envs/chatbot_temp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# 测试Embedding生成
text = "test"
vec = embeddings.embed_query(text)
print(len(vec))  # 输出向量维度，如 text-embedding-3-small: 1536、bge-large-zh-v1.5: 1024

1024


In [16]:
from langchain_community.vectorstores import FAISS

# 存储到FAISS向量库
vectorstore = FAISS.from_documents(splits, embeddings)

In [17]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # 返回top3结果

In [18]:
from langchain_core.prompts import ChatPromptTemplate

template = """基于以下上下文回答问题：
{context}

问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [19]:
# from langchain_openai import ChatOpenAI

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")

# model = ChatOpenAI(
#     api_key=OPENAI_API_KEY,
#     base_url=OPENAI_BASE_URL,  # 自定义API地址
#     model="gpt-4-turbo"
# )

from langchain_ollama import ChatOllama

# 初始化本地Ollama模型
llm = ChatOllama(
    model="qwen2.5:32b",
    temperature=0.1,
    base_url="http://localhost:11434"
)

In [20]:

from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 构建RAG链
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [21]:
response = rag_chain.invoke("如何复制conda环境？")
print(response)

要复制一个conda环境，可以使用`conda create`命令，并加上`--clone`选项。以下是一个例子：

假设你有一个名为`py36`的环境，你想基于这个环境创建一个新的副本叫做`py36_bak`，你可以运行如下命令：

```bash
conda create -n py36_bak --clone py36
```

这条命令会复制`py36`环境中所有的包和依赖到新创建的名为`py36_bak`的环境里。


In [22]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

partial_chain = RunnableParallel( 
    {"context": retriever, "question": RunnablePassthrough()}
)
response = partial_chain.invoke("如何复制conda环境？")
print(response)

{'context': [Document(id='1c2269c2-e880-425b-8a09-161f0ec7fae4', metadata={'source': 'ee04ec75c67b460b9aa0b2143cead3a2'}, page_content='conda env remove -p 要删除的虚拟环境路径\n根据environment.yaml安装conda环境\n比如splatam的代码仓库里给了个environment.yaml\nname: splatam\nchannels:\n  - pytorch\n  - conda-forge\n  - defaults\ndependencies:\n  - cudatoolkit=11.6\n  - python=3.10\n  - pytorch=1.12.1\n  - torchaudio=0.12.1\n  - torchvision=0.13.1\n  - tqdm=4.65.0\n  - Pillow\n  - faiss-gpu\n  - opencv\n  - imageio\n  - matplotlib\n  - kornia\n  - natsort\n  - pyyaml\n  - wandb\n  - pip\n  - pip:\n    - lpips\n    - open3d==0.16.0\n    - torchmetrics\n    - cyclonedds\n    - git+https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth/tree/cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110\n# 先退出所有环境\nconda deactivate\n# 根据environment.yaml安装指定环境\nconda env create -f environment.yml -v\n其中channels指定了包的搜索来源，比如cudatoolkit需要conda-forge\nconda install -c conda-forge cudatoolkit=11.6 # 指定channels，否则defaults channel中是

```json
{
    'context': [
        Document(
            id='1c2269c2-e880-425b-8a09-161f0ec7fae4', 
            metadata={'source': 'ee04ec75c67b460b9aa0b2143cead3a2'}, 
            page_content='conda env remove -p 要删除的虚拟环境路径\n根据environment.yaml安装conda环境\n比如splatam的代码仓库里给了个environment.yaml\nname: splatam\nchannels:\n  - pytorch\n  - conda-forge\n  - defaults\ndependencies:\n  - cudatoolkit=11.6\n  - python=3.10\n  - pytorch=1.12.1\n  - torchaudio=0.12.1\n  - torchvision=0.13.1\n  - tqdm=4.65.0\n  - Pillow\n  - faiss-gpu\n  - opencv\n  - imageio\n  - matplotlib\n  - kornia\n  - natsort\n  - pyyaml\n  - wandb\n  - pip\n  - pip:\n    - lpips\n    - open3d==0.16.0\n    - torchmetrics\n    - cyclonedds\n    - git+https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth/tree/cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110\n# 先退出所有环境\nconda deactivate\n# 根据environment.yaml安装指定环境\nconda env create -f environment.yml -v\n其中channels指定了包的搜索来源，比如cudatoolkit需要conda-forge\nconda install -c conda-forge cudatoolkit=11.6 # 指定channels，否则defaults channel中是找不到cudatoolkit的\n如果“conda env create -f environment.yml -v”速度慢，有加速的办法，比如用mamba，更简单的就是升级conda版本\n$ conda --version\n\tconda 23.11.0\n$ conda env create -f environment.yml -v'), 
        Document(
            id='dd28dba3-9685-4f01-aa56-47329fbc2fed', 
            metadata={'source': 'ee04ec75c67b460b9aa0b2143cead3a2'}, 
            page_content='$ python test.py \n4.10.0\n/home/hey/anaconda3/envs/test/lib/python3.8/site-packages/cv2.cpython-38-x86_64-linux-gnu.so\n# 验证一下，完成任务了\nubuntu 2204安装conda\n下载最新当前最新版本的conda\nconda config --set auto_activate_base false\nconda init --reverse $SHELL\nubuntu 20.04 安装 conda\n下载最新当前（20230520）最新版本的conda：\nAnaconda3-2023.03-1-Linux-x86_64.sh\nhttps://mirrors.tuna.tsinghua.edu.cn/anaconda/archive/Anaconda3-2023.07-1-Linux-x86_64.sh\nbash Anaconda3-2021.11-Linux-x86_64.sh\n安装位置在：/home/hey/anaconda3\n自动初始化？yes\n（如果选择yes，则每次打开新的终端时，都会默认进入到base环境）\n升级conda（解决DEBUG:urllib3.connectionpool的问题）\n旧版本conda执行安装命令时总是会报DEBUG:urllib3.connectionpool:Starting new HTTPS conne\n解决办法就是升级conda\nconda update conda\n删除指定的conda env\n# 退出当前环境\nconda deactivate\n# 列出当前环境\nconda env list\n# 通过环境名来删除指定环境\nconda remove -n 需要删除的环境名 --all\n# 通过环境的路径删除指定环境\nconda env remove -p 要删除的虚拟环境路径\n根据environment.yaml安装conda环境\n比如splatam的代码仓库里给了个environment.yaml\nname: splatam\nchannels:\n  - pytorch\n  - conda-forge\n  - defaults\ndependencies:\n  - cudatoolkit=11.6'), 
        Document(
            id='632dd147-cffb-4d5c-ba25-12fe19581070', 
            metadata={'source': 'ee04ec75c67b460b9aa0b2143cead3a2'}, 
            page_content='通过environment.yaml环境文件创建文件： conda env create -f environment.yaml\n查看已安装的包：conda list\n搜索包：\n模糊搜索：conda search <package_name1>\nconda list | findstr torch\nconda search --full-name <package name> 搜索指定的包\n比如 conda search --full-name tensorflow 显示所有的tensorflow包。\n安装包：conda install <package_name1> <package_name2>\n卸载包：\nconda remove <package_name>\nconda remove --name [env_name] --all，回车后出现一系列在这个环境所安装的包；输入【y】进行环境的删除。\nconda uninstall package_name(包名)\ncopy环境：\nconda create\n --\nname 新环境名\n --\nclone 旧环境名\n检查更新当前的conda版本\nconda update conda\n参考\nv2calibration代码环境配置（conda）\nwin10 conda环境中conda无法安装opencv-python，以及pip失效问题\n在win的cmd中安装opencv-python和opencv-contrib-python可真气死我了\n进入conda虚拟环境\nconda search opencv-python\n无，查看一下源，有清华源，没问题。但conda里是有opencv的\n这个能用不？\npip search opencv-python\n懵逼，，\n继续懵逼\n翻不翻墙都不行\n谷歌了一下，有两个解决办法\n临时办法：\npip install opencv-python==3.4.15.55 -i \nhttp://pypi.douban.com/simple\n --trusted-host \npypi.douban.com\n临时走一下豆瓣的源\n永久办法：\nref：\n[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\n[install]'), 
        Document(
            id='ee3c4b11-c889-413e-bfcb-48931ad44b8a', 
            metadata={'source': 'ee04ec75c67b460b9aa0b2143cead3a2'}, 
            page_content='conda config --set proxy_servers.https https://proxy_server:proxy_port\npip常用命令\npip list 查看以及安装的包和版本号\npip --version：查看已经安装了的pip版本\npip install -U pip：升级pip\npip list 或 pip freeze：查看当前已经安装好了包及版本\npip install package_name(包名)：下载安装包\npip uninstall package_name(包名)： 卸载安装包\npip show package_name(包名)：显示安装包信息（安装路径、依赖关系等）\nconda常用命令 \n创建环境：conda create -n <env_name> <packages>\n基于python3.6创建一个名为py36的环境\nconda create --name py36 python=3.6\n激活环境：conda activate <env_name>\n退出环境：conda deactivate <env_name>\n查看已安装的环境信息：\nconda env list\nconda info -e \n$ pip list\n复制环境：conda create -n <new_env_name> --clone <origin_env_name>\n通过克隆py36来创建一个称为py36_bak的副本：\nconda create -n py36_bak --clone py36\n删除环境：\nconda env remove -n <env_name>\n保存环境信息到environment.yaml文件中：conda env export > environment.yaml\n通过environment.yaml环境文件创建文件： conda env create -f environment.yaml\n查看已安装的包：conda list\n搜索包：\n模糊搜索：conda search <package_name1>\nconda list | findstr torch\nconda search --full-name <package name> 搜索指定的包'), 
        Document(
            id='a97584d4-78f1-4ee0-9b80-9aab91d7d02c', 
            metadata={'source': 'ee04ec75c67b460b9aa0b2143cead3a2'}, 
            page_content='如果“conda env create -f environment.yml -v”速度慢，有加速的办法，比如用mamba，更简单的就是升级conda版本\n$ conda --version\n\tconda 23.11.0\n$ conda env create -f environment.yml -v\n\tChannels:\n\t - pytorch\n\t - conda-forge\n\t - defaults\n\tPlatform: linux-64\n\tCollecting package metadata (repodata.json): ...working... info     libmamba Reading cache files \'/tmp/tmpv4iityir.json.*\' for repo index \'installed\'\n\tinfo     \nlibmamba\n Reading repodata.json file "/tmp/tmpv4iityir.json" for repo installed\n\tinfo     \nlibmamba\n Reading cache files \'/home/hey/anaconda3/pkgs/cache/ee0ed9e9.json.*\' for repo index \'https://conda.anaconda.org/pytorch/linux-64\'\n\t...\n23.11.0 已经自动用上manba了\n删除conda环境中的指定包\nconda remove --name $your_env_name $package_name\npip 换国内源（加速）\n用国外源有两个缺点：费流量、shell中让conda pip走代理，还得设置\npip 临时走一下国内源：（有时某个源找不到这个包或下载速度慢，就换另一个源）\n# 清华源\npip install opencv-python==3.4.15.55 -i https://pypi.tuna.tsinghua.edu.cn/simple\n# 豆瓣源\npip install opencv-python==3.4.15.55 -i \nhttp://pypi.douban.com/simple\n --trusted-host \npypi.douban.com')
        ], 
    'question': '如何复制conda环境？'
}
```

document3和4都命中了question

In [25]:
# 无参考资料时，gpt的回复：

response = llm.invoke("如何复制conda环境？")
print(response.content)

在使用 Conda 管理 Python 开发环境时，有时需要复制一个已有的环境以保持一致性或进行实验。以下是复制 Conda 环境的步骤：

1. **列出所有环境**：首先可以查看当前所有的 Conda 环境。
   ```bash
   conda env list
   ```
   或者使用简写形式：
   ```bash
   conda info --envs
   ```

2. **导出环境配置文件**：选择一个要复制的环境，然后将其包列表导出到一个 YAML 文件中。假设你要复制名为 `my_env` 的环境。
   ```bash
   conda env export --name my_env > environment.yml
   ```
   这个命令会生成一个 `environment.yml` 文件，其中包含了该环境中所有已安装的包及其版本信息。

3. **创建新环境**：使用导出的 YAML 文件来创建一个新的 Conda 环境。假设你想将新的环境命名为 `my_env_copy`。
   ```bash
   conda env create --name my_env_copy --file environment.yml
   ```

4. **验证新环境**：你可以通过激活新环境并列出其包来检查是否成功复制了原环境。
   ```bash
   conda activate my_env_copy
   conda list
   ```

以上步骤将帮助你完整地复制一个 Conda 环境。注意，如果原环境中包含一些非 PyPI 源的包（例如从 Anaconda 仓库），确保你的新环境也能访问这些源。

另外，如果你只需要快速复制环境而不关心特定版本号，可以使用 `--from-history` 参数来导出环境配置文件：
```bash
conda env export --name my_env --from-history > environment.yml
```
这种方式会忽略一些非必需的包依赖关系，只保留那些通过显式命令安装的包。


总结，对于复制conda环境的问题，rag和非rag给了不同的角度的回答，rag的回答是clone，非rag回答是export后create，虽然笔记里这两种的命令都给了。也许应该测试有时效性或者个人问题，而非通用问题。
<img src="./assets/截图 2025-03-13 15-41-06.png" alt="运行截图" width="30%">